In [1]:
import pandas as pd
from pathlib import Path

# ==== INPUTS ====
# Weather CSV we just created in step 7
wx_path = Path("laguardia_weather_2022.csv")

# EITHER: your already-merged trips file (fastest)
trips_path = Path("citibike_2022.parquet")  # <- change to .csv if that's what you saved

# OR: uncomment this block to build from monthly raw files in extracted/ (slower)
# monthly = sorted(Path("extracted/2022-citibike-tripdata").glob("2022??-citibike-tripdata.csv"))
# if not monthly:
#     raise FileNotFoundError("No monthly files found in extracted/2022-citibike-tripdata/")
# dfs = [pd.read_csv(f, low_memory=False) for f in monthly]
# rides = pd.concat(dfs, ignore_index=True)

# ---- Load rides (prefer your single file if available) ----
if trips_path.suffix == ".parquet" and trips_path.exists():
    rides = pd.read_parquet(trips_path)
elif trips_path.suffix == ".csv" and trips_path.exists():
    rides = pd.read_csv(trips_path, low_memory=False)
else:
    raise FileNotFoundError("Could not find your combined trips file. Point trips_path to it.")

# ---- Basic cleaning + daily aggregation ----
# Ensure datetime
rides["started_at"] = pd.to_datetime(rides["started_at"], errors="coerce")
rides["ended_at"]   = pd.to_datetime(rides["ended_at"],   errors="coerce")

# Filter bad rows
rides = rides.dropna(subset=["started_at", "ended_at"])

# Duration in minutes
rides["duration_min"] = (rides["ended_at"] - rides["started_at"]).dt.total_seconds() / 60

# Day column
rides["date"] = rides["started_at"].dt.date

# Daily metrics
daily = (
    rides.groupby("date")
         .agg(
             trips=("ride_id", "count"),
             mean_duration_min=("duration_min", "mean"),
             member_trips=("member_casual", lambda s: (s == "member").sum() if s.notna().any() else 0),
             casual_trips=("member_casual", lambda s: (s == "casual").sum() if s.notna().any() else 0)
         )
         .reset_index()
)

# ---- Load weather and merge ----
wx = pd.read_csv(wx_path, parse_dates=["date"])
wx["date"] = wx["date"].dt.date  # to pure date to match

merged = pd.merge(daily, wx, on="date", how="left").sort_values("date")

# Optional: re-order columns if present
col_order = [c for c in ["date","trips","member_trips","casual_trips","mean_duration_min","TMAX","TMIN","PRCP","SNOW","AWND"] if c in merged.columns]
merged = merged[col_order + [c for c in merged.columns if c not in col_order]]

# Save small, analysis-ready CSV (365 rows)
out_csv = "citibike_weather_2022.csv"
merged.to_csv(out_csv, index=False)
merged.head(), merged.shape, out_csv


(         date  trips  member_trips  casual_trips  mean_duration_min  TMAX  \
 0  2021-01-30      1             0             1      687646.693117   NaN   
 1  2021-02-15      1             0             1      779090.164883   NaN   
 2  2021-03-11      1             0             1      664503.451733   NaN   
 3  2021-03-14      1             1             0      468320.034500   NaN   
 4  2021-03-31      1             0             1      781858.025617   NaN   
 
    TMIN  PRCP  SNOW  AWND  
 0   NaN   NaN   NaN   NaN  
 1   NaN   NaN   NaN   NaN  
 2   NaN   NaN   NaN   NaN  
 3   NaN   NaN   NaN   NaN  
 4   NaN   NaN   NaN   NaN  ,
 (402, 10),
 'citibike_weather_2022.csv')

- Loads your 2022 CitiBike trips, parses timestamps, computes trip duration, and creates a daily table (trips, mean duration, member vs casual counts).
- Loads LaGuardia weather (daily) and left-joins on date.
- Exports a small, analysis-ready file: citibike_weather_2022.csv.